# Using whisper to transcribe audio

In [1]:
!pip install git+https://github.com/openai/whisper.git
# !sudo apt update && sudo apt install ffmpeg   #run in terminal

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-diky5nd3
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-diky5nd3
  Resolved https://github.com/openai/whisper.git to commit c0d2f624c09dc18e709e37c2ad90c039a4eb72a2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.5/170.5 MB 6.3 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20250625-py3-none-any.whl size=803979 sha256=995ae7bc6fc905123d43268ed1d3c10cbd9781578fefdce56c653ad5172a0960
  Stored in directory: /tmp/pip-ephem-wheel-cache-ux39m03d/wheels/c3/03/25/5e0ba78bc27a3a089f137c9f1d92fdfce16d06996c071a016c
Successfully built openai-whisper


In [3]:
!whisper -h

usage: whisper [-h] [--model MODEL] [--model_dir MODEL_DIR] [--device DEVICE]
               [--output_dir OUTPUT_DIR]
               [--output_format {txt,vtt,srt,tsv,json,all}]
               [--verbose VERBOSE] [--task {transcribe,translate}]
               [--language {af,am,ar,as,az,ba,be,bg,bn,bo,br,bs,ca,cs,cy,da,de,el,en,es,et,eu,fa,fi,fo,fr,gl,gu,ha,haw,he,hi,hr,ht,hu,hy,id,is,it,ja,jw,ka,kk,km,kn,ko,la,lb,ln,lo,lt,lv,mg,mi,mk,ml,mn,mr,ms,mt,my,ne,nl,nn,no,oc,pa,pl,ps,pt,ro,ru,sa,sd,si,sk,sl,sn,so,sq,sr,su,sv,sw,ta,te,tg,th,tk,tl,tr,tt,uk,ur,uz,vi,yi,yo,yue,zh,Afrikaans,Albanian,Amharic,Arabic,Armenian,Assamese,Azerbaijani,Bashkir,Basque,Belarusian,Bengali,Bosnian,Breton,Bulgarian,Burmese,Cantonese,Castilian,Catalan,Chinese,Croatian,Czech,Danish,Dutch,English,Estonian,Faroese,Finnish,Flemish,French,Galician,Georgian,German,Greek,Gujarati,Haitian,Haitian Creole,Hausa,Hawaiian,Hebrew,Hindi,Hungarian,Icelandic,Indonesian,Italian,Japanese,Javanese,Kannada,Kazakh,Khmer,Korean,Lao,L

In [4]:
!whisper "/content/testaudio1.wav" --model medium.en

100%|█████████████████████████████████████| 1.42G/1.42G [00:42<00:00, 35.7MiB/s]
/usr/local/lib/python3.12/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
[00:00.000 --> 00:05.560]  Hello, this is a test, I want to see if this works.


#Test recording audio


In [6]:
from google.colab import output
output.enable_custom_widget_manager()

In [5]:
!pip install ipywidgets

import ipywidgets as widgets
from IPython.display import display, Audio
from google.colab import output
import numpy as np
import io
import base64

# Create start and stop buttons
start_button = widgets.Button(description="Start Recording")
stop_button = widgets.Button(description="Stop Recording")

recording = False

# Function to start recording
def start_recording(b):
    global recording
    recording = True
    output.eval_js('startRecording()')
    print("Recording started...")

# Function to stop recording and save the file
def stop_recording(b):
    global recording
    if not recording:
        print("No recording in progress.")
        return

    recording = False
    audio_data = output.eval_js('stopRecording()')

    if audio_data is None:
        print("No audio data recorded.")
        return

    # Decode base64 data
    audio_bytes = base64.b64decode(audio_data['audio'])

    # Create and display Audio widget
    audio_obj = Audio(data=audio_bytes, rate=audio_data['sampleRate'])
    display(audio_obj)

    # Save as WAV file
    with open("recorded_audio.wav", "wb") as f:
        f.write(audio_bytes)

    print("Audio saved as 'recorded_audio.wav'")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.9 MB/s eta 0:00:00


In [8]:
#record invoice
# Connect buttons to functions
start_button.on_click(start_recording)
stop_button.on_click(stop_recording)

# Display buttons
display(start_button, stop_button)

# JavaScript to handle audio recording
output.eval_js('''
    let recorder, stream;

    async function startRecording() {
        stream = await navigator.mediaDevices.getUserMedia({ audio: true });
        recorder = new MediaRecorder(stream);
        chunks = [];
        recorder.ondataavailable = e => chunks.push(e.data);
        recorder.start();
    }

    function stopRecording() {
        return new Promise(resolve => {
            recorder.onstop = () => {
                const blob = new Blob(chunks, { type: 'audio/wav' });
                chunks = [];
                stream.getTracks().forEach(track => track.stop());
                const reader = new FileReader();
                reader.onload = () => {
                    const base64data = reader.result.split(',')[1];
                    resolve({audio: base64data, sampleRate: 44100});
                };
                reader.readAsDataURL(blob);
            };
            recorder.stop();
        });
    }
''')

Button(description='Start Recording', style=ButtonStyle())

Button(description='Stop Recording', style=ButtonStyle())

Recording started...


Audio saved as 'recorded_audio.wav'
